In [1]:
!pip install transformers
import torch
from tqdm.auto import tqdm
from transformers import AdamW
from transformers import AutoTokenizer, BertForMaskedLM

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

#change sheetname
# train_df=pd.read_excel('../../data/data.xlsx', sheet_name=0)
test_df=pd.read_excel('../../data/data.xlsx', sheet_name=2)
val_df=pd.read_excel('../../data/data.xlsx', sheet_name=1)


bad_chars = [';', ':', "(", ")", "-", "'", "\"", "_", ".", ",", " "]

def components(df):
    statements = list(df['statements'])
    labels = list(df['labels'])
    genes = list(df['target_genes'])
    return statements, labels, genes


In [3]:
train_df.head()

NameError: name 'train_df' is not defined

In [4]:
test_df.head()

,Unnamed: 0,target_genes,pmcid,statements,review_label,label,Remarks,labels,connor_notes
0,1436,msbA,4979069,coli conclude that the conformational transiti...,-1.0,1,ABC transporter,0,NaN
1,343,CpxR,5063474,"The results showed that, apart from the previo...",-1.0,-1,NaN,0,NaN
2,1382,mphI,5760710,MphI shares high sequence identity (94%) to ho...,1.0,1,NaN,0,Note that the actual conclusion of the linked ...
3,187,cpxA,3319533,The disruption at cpxAR operon was confirmed w...,-1.0,1,plasmid,0,NaN
4,1854,sdiA,2812512,It was recently discovered that S . Typhimuriu...,-1.0,-1,NaN,0,NaN


In [4]:
val_df.head()

,target_genes,statements,labels
0,NDM-1,5% in Indian and Pakistan hospitals . In addit...,1
1,otrC,"Additionally, the significantly enhanced vanco...",1
2,FosB,"Thus, the P1 space group appears to be the res...",1
3,CpxR,"In Klebsiella pneumoniae, CpxR is involved in ...",1
4,mfd,"3) . Thus, the reduced spontaneous mutation ra...",1


In [5]:
# train_statements, train_labels, train_genes=components(train_df)
test_statements, test_labels, test_genes=components(test_df)
val_statements, val_labels, val_genes=components(val_df)

In [6]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 4 GPU(s) available.
Device name: Quadro RTX 8000


In [7]:
torch.cuda.set_device(2)

In [8]:
import random
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
set_seed(42)    # Set seed for reproducibility

In [9]:
prompt_template_encoding=tokenizer.tokenize(" is antibiotic resistant ")
print(prompt_template_encoding)

['is', 'antibiotic', 'resistant']


In [10]:
!nvidia-smi

Mon Apr  4 18:22:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:3D:00.0 Off |                  Off |
| 33%   32C    P8    35W / 260W |      3MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:3F:00.0 Off |                  Off |
| 33%   

In [12]:
#prepare mask keywords

bad_chars = [';', ':', "(", ")", "-", "'", "\"", "_", ".", ",", " "]

rule1=['resistan', 'efflux']

rule2=["beta-lactamase", "beta lactamase",  "streptogramin inactivation enzyme", 
"fosfomycin inactivation enzyme",
"tetracycline inactivation enzyme ", #AMR gene
"macrolide inactivation enzyme",
"rifampin inactivation enzyme",
"aminoglycoside acetyltransferase", #"AAC",
"chloramphenicol phosphotransferase",
"aminoglycoside phosphotransferase", #"APH",
"chloramphenicol acetyltransferase", #"CAT", 
"aminoglycoside nucleotidyltransferase", #"ANT",
"lincosamide nucleotidyltransferase", #"LNU" 
"streptothricin acetyltransferase", #"SAT", 
"fusidic acid inactivation enzyme",
"Edeine acetyltransferase", 
"viomycin phosphotransferase", 
"ciprofloxacin phosphotransferase",
"Bah amidohydrolase",
"MDR",
"MRSA"]

rule3=['bla', 'mec']

rule4=['MIC', 'increase', 'fold']

antibiotics_df=pd.read_excel('../../data/antibiotics.xlsx', sheet_name=0)
temp_keywords=list(antibiotics_df['antibiotics'])
temp_keywords = [y for x in [temp_keywords, rule1] for y in x]
temp_keywords = [y for x in [temp_keywords, rule2] for y in x]
temp_keywords = [y for x in [temp_keywords, rule3] for y in x]
temp_keywords = [y for x in [temp_keywords, rule4] for y in x]
mask_keywords=[]
for i in range(len(temp_keywords)):
    keywords=temp_keywords[i].replace("-", " ").split()
#     for j in range(len(keywords)):
#         filtered_word=''.join(k for k in keywords[j] if not k in bad_chars)
    mask_keywords = [y for x in [mask_keywords, keywords] for y in x]
mask_keywords=[i for i in mask_keywords if len(i)>2]

In [13]:
# mask_keywords=random.sample(mask_keywords, int(len(mask_keywords)*0.50))

In [14]:
def preprocessing_for_bert_dev(statements, labels, genes):
    pretokenized_encodings=[]
    mask_indices=[]
    prompt_encodings=[]
    all_masks=[]
    for idx in range(len(statements)):
        sub_tokens=tokenizer.tokenize(statements[idx], add_special_tokens=True, truncation=True, max_length=512, return_tensors='pt')
        gene_encoding=tokenizer.tokenize(genes[idx])
        prompt=[]
        prompt = [y for x in [prompt, gene_encoding] for y in x]
        prompt = [y for x in [prompt, prompt_template_encoding] for y in x]
        if(labels[idx]==1):
            prompt.append('true')
        else:
            prompt.append('false')
        if(len(sub_tokens)+len(prompt)+1>512):
            sub_tokens=sub_tokens[:(512-len(prompt)-2)]
            sub_tokens.append('[SEP]')
        sub_tokens = [y for x in [sub_tokens, prompt] for y in x]
        mask_index=len(sub_tokens)-1
        mask_indices.append(mask_index)
        prompt_encodings.append(prompt)
        
    return prompt_encodings, mask_indices

In [15]:
# # #prompt, label_mask, rule_mask
# train_pretokenized_encodings, train_mask_indices=preprocessing_for_bert_dev(train_statements, train_labels, train_genes)
# test_pretokenized_encodings, test_mask_indices, test_rule_masks=preprocessing_for_bert(test_statements, test_labels, test_genes)

# train_pretokenized_encodings, mask_indices=preprocessing_for_bert(statements, labels, genes)
val_pretokenized_encodings, val_mask_indices=preprocessing_for_bert_dev(val_statements, val_labels, val_genes)
test_pretokenized_encodings, test_mask_indices=preprocessing_for_bert_dev(test_statements, test_labels, test_genes)

In [16]:
#prompt, label_mask, rule_mask
# train_pretokenized_strings=[]
# for idx in range(len(train_pretokenized_encodings)):
#     train_pretokenized_strings.append(tokenizer.convert_tokens_to_string(train_pretokenized_encodings[idx]))

val_pretokenized_strings=[]
for idx in range(len(val_pretokenized_encodings)):
    val_pretokenized_strings.append(tokenizer.convert_tokens_to_string(val_pretokenized_encodings[idx]))

test_pretokenized_strings=[]
for idx in range(len(test_pretokenized_encodings)):
    test_pretokenized_strings.append(tokenizer.convert_tokens_to_string(test_pretokenized_encodings[idx]))

In [17]:
def tokenize_normal(statements, prompts, mask_ind):
    inputs = tokenizer(
        statements,prompts,
        max_length=512,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='pt'
    )
    inputs['labels']=torch.ones(len(inputs['input_ids']),512, dtype=int)    
    inputs['mask_idx']=torch.zeros(len(inputs['input_ids']),1, dtype=int)
    for idx in range(len(inputs['input_ids'])):
        inputs['labels'][idx][mask_ind[idx]]=inputs['input_ids'][idx][mask_ind[idx]]
        inputs['input_ids'][idx][mask_ind[idx]]=tokenizer.mask_token_id
        #################
        ## MLM ##
#         rule_mask=rule_masks[idx]
#         for i in range(len(rule_mask)):
#             if rule_mask[i]==1:
#                 inputs['labels'][idx][i]=inputs['input_ids'][idx][i]
#                 inputs['input_ids'][idx][i]=tokenizer.mask_token_id
        #################
        inputs['mask_idx'][idx][0]=mask_ind[idx]
    return inputs

In [18]:
class BookDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        labels = self.encodings['labels'][index]
        attention_mask = self.encodings['attention_mask'][index]
        token_type_ids = self.encodings['token_type_ids'][index]
        mask_idx = self.encodings['mask_idx'][index]
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'mask_idx': mask_idx
        }

In [19]:
# train_inputs_normal=tokenize_normal(train_statements, train_pretokenized_strings, train_mask_indices)
val_inputs_normal=tokenize_normal(val_statements, val_pretokenized_strings, val_mask_indices)
test_inputs_normal=tokenize_normal(test_statements, test_pretokenized_strings, test_mask_indices)

In [21]:
# import pickle
# ftr = open('final_train_inputs_new_prompt2_MLM_50.txt', 'wb')
# pickle.dump(train_inputs, ftr)
# ftr.close()
# # fv = open('val_inputs_prompt1_MLM.txt', 'wb')
# # pickle.dump(val_inputs, fv)
# # fv.close()
# # ft = open('test_inputs_prompt1_MLM.txt', 'wb')
# # pickle.dump(test_inputs, ft)
# # ft.close()

In [22]:
# import pickle
# with open('final_train_inputs_new_prompt2_MLM_50.txt', 'rb') as handle:
#     data = handle.read()
# train_inputs_MLM = pickle.loads(data)

# # with open('val_inputs_prompt1_MLM.txt', 'rb') as handle:
# #     data = handle.read()
# # val_inputs_MLM = pickle.loads(data)

# # with open('test_inputs_prompt1_MLM.txt', 'rb') as handle:
# #     data = handle.read()
# # test_inputs_MLM = pickle.loads(data)

In [23]:
# ###################
# train_dataset=BookDataset(train_inputs)
# ####################

In [20]:
# train_dataset_normal=BookDataset(train_inputs_normal)
val_dataset=BookDataset(val_inputs_normal)
test_dataset=BookDataset(test_inputs_normal)
# train_dataset=BookDataset(train_inputs_MLM)

In [21]:
import torch.nn.functional as F

In [22]:
# def train(model, train_dataloader):

#     set_seed(42)
#     optimizer = AdamW(model.parameters(), lr=2e-05)
#     model.train()

#     # Tracking variables
#     train_loss = []
#     tp=0
#     tn=0
#     fp=0
#     fn=0
#     unk_pos=0
#     unk_neg=0
#     # for epoch in range(epochs):
#     #     loop = tqdm(train_dataloader)
#     for batch in train_dataloader:
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         labels = batch['labels'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss   
#         train_loss.append(loss.item())
#         b_probabilities = F.softmax(outputs.logits, dim=-1)
#         # Get the predictions
#         preds = torch.argmax(b_probabilities, dim=2)
#     #     print("preds: ", preds[0])
#     #     print("labels: ", labels[0])
#         # Calculate the accuracy rate
#         for i in range(len(batch['input_ids'])):
#             pred=preds[i][batch['mask_idx'][i][0]]
#             label=labels[i][batch['mask_idx'][i][0]]
# #             print("pred: ", tokenizer.convert_ids_to_tokens(pred.item()))
# #             print("label: ", tokenizer.convert_ids_to_tokens(label.item()))
#             if pred == 6228 and label == 6228: 
#                 tp+=1
#             elif pred == 6228 and label == 7198:
#                 fp+=1
#             elif pred == 7198 and label == 7198:
#                 tn+=1
#             elif pred == 7198 and label == 6228:
#                 fn+=1
#             elif label == 6228:
#                 unk_pos+=1
#             else:
#                 unk_neg+=1

#         loss.backward()
#         optimizer.step()


# #     train_loss = np.mean(train_loss)
# #     #balanced accuracy = (tpr+tnr)/2
# #     train_precision=tp/(tp+fp)
# #     train_recall=tp/(tp+fn+unk_pos)
# #     train_f1=2*train_precision*train_recall/(train_precision+train_recall)
# #     train_balanced_accuracy = (train_recall+(tn/(tn+fp+unk_neg)))/2
# #     train_accuracy = (tp+tn)/(tp+tn+fp+fn+unk_pos+unk_neg)
#     train_total_not_predicted = unk_pos+unk_neg
    
    
#     train_loss = np.mean(train_loss)
# #     train_accuracy = np.mean(train_accuracy)
    
#     beta=0.5
#     train_precision_pos=tp/(tp+fp)
#     train_recall_pos=tp/(tp+fn+unk_pos)
#     train_f1_pos=2*train_precision_pos*train_recall_pos/(train_precision_pos+train_recall_pos)
# #     train_balanced_accuracy = (train_recall+(tn/(tn+fp)))/2
#     train_precision_neg=tn/(tn+fn)
#     train_recall_neg=tn/(tn+fp+unk_neg)
#     train_f1_neg=2*train_precision_neg*train_recall_neg/(train_precision_neg+train_recall_neg)
#     train_f1_beta_pos=((1+beta*beta)*train_precision_pos*train_recall_pos)/((1+beta*beta)*train_precision_pos*train_recall_pos+beta*beta*train_precision_pos+train_recall_pos)
    
#     train_accuracy = (tp+tn)/(tp+tn+fp+fn+unk_pos+unk_neg)
# #     train_precision_neg=tn/(tn+fn)
# #     train_recall_neg=tn/(tn+fp)
# #     train_f1_neg=2*train_precision_pos*train_recall_pos/(train_precision_pos+train_recall_pos)
#     train_f1_beta_neg=((1+beta*beta)*train_precision_neg*train_recall_neg)/((1+beta*beta)*train_precision_neg*train_recall_neg+beta*beta*train_precision_neg+train_recall_neg)
    
#     train_f1_beta=(train_f1_beta_pos+train_f1_beta_neg)/2
#     print(f" check_train_accuracy: {(tp+tn)/(tp+tn+fp+fn)}")

#     return [train_loss, train_accuracy, train_precision_pos, train_recall_pos, train_f1_pos, train_f1_beta_pos,train_precision_neg, train_recall_neg, train_f1_neg, train_f1_beta_neg, train_f1_beta, train_total_not_predicted]

# #     return [train_loss, train_precision, train_recall, train_f1, train_balanced_accuracy, train_accuracy, train_total_not_predicted] 

#     #note total=11264


In [22]:
# Specify loss function
# loss_fn = torch.nn.CrossEntropyLoss()
# model.to(device)
def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
#     val_accuracy = []
    val_loss = []
    pred_label=[]
    tp=0
    tn=0
    fp=0
    fn=0
    unk_pos=0
    unk_neg=0
    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids = batch['input_ids'].to(device)
        b_labels = batch['labels'].to(device)
        b_attn_mask = batch['attention_mask'].to(device)
        # Compute logits
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_attn_mask, labels=b_labels)

        # Compute loss
#         loss=torch.nn.functional.cross_entropy(outputs.logits.view(-1, tokenizer.vocab_size), b_labels.view(-1))
        loss=outputs.loss
#         loss = loss_fn(outputs.logits, b_labels)
        val_loss.append(loss.item())
        b_probabilities = F.softmax(outputs.logits, dim=-1)
        
#         pred = torch.argmax( scores[0][7]).item()
#         print("predicted token:", pred, tokenizer.convert_ids_to_tokens([pred])  )
#         print(NLLLos( logSoftmax(torch.unsqueeze(scores[0][7], 0)), torch.tensor([pred]))) #the same as F.cross_entropy(scores.view(-1, tokenizer.vocab_size), labels.view(-1))
        
        # Get the predictions
        preds = torch.argmax(outputs.logits, dim=2)
#         print("preds: ", preds[0])
#         print("labels: ", b_labels[0])
        # Calculate the accuracy rate
#         for i in batch:
        for i in range(len(batch['input_ids'])):
            pred=preds[i][batch['mask_idx'][i][0]]
            label=b_labels[i][batch['mask_idx'][i][0]]
#             print("pred: ", preds[i])
#             print("label: ", labels[i])
            prob.append(b_probabilities[i][batch['mask_idx'][i][0]][6228].item())
            if pred == 6228 and label == 6228: 
                tp+=1
                pred_label.append(1)
#                 prob_tp.append(b_probabilities[i][batch['mask_idx'][i][0]][6228].item())
            elif pred == 6228 and label == 7198:
                fp+=1
                pred_label.append(1)
#                 prob_fp.append(b_probabilities[i][batch['mask_idx'][i][0]][6228].item())
            elif pred == 7198 and label == 7198:
                tn+=1
                pred_label.append(0)
            elif pred == 7198 and label == 6228:
                fn+=1    
                pred_label.append(0)
            elif label == 6228:
                unk_pos+=1
                pred_label.append(1)
            else:
                unk_neg+=1
                pred_label.append(0)
#         accuracy = (preds == b_labels).cpu().numpy().mean() * 100
#         val_accuracy.append(accuracy)
    val_total_not_predicted = unk_pos+unk_neg
    # Compute the average accuracy and loss over the validation set.
    beta=0.5
    val_loss = np.mean(val_loss)
#     val_accuracy = np.mean(val_accuracy)
    try:
        val_precision_pos=tp/(tp+fp)
    except ZeroDivisionError:
        val_precision_pos=0
    try:
        val_recall_pos=tp/(tp+fn)
    except ZeroDivisionError:
        val_recall_pos=0
    try:
        val_f1_pos=2*val_precision_pos*val_recall_pos/(val_precision_pos+val_recall_pos)
    except ZeroDivisionError:
        val_f1_pos=0
    try:
        val_f1_beta_pos=((1+beta*beta)*val_precision_pos*val_recall_pos)/((1+beta*beta)*val_precision_pos*val_recall_pos+beta*beta*val_precision_pos+val_recall_pos)
    except ZeroDivisionError:
        val_f1_beta_pos=0
#     try:
#         val_balanced_accuracy = (val_recall+(tn/(tn+fp)))/2
#     except ZeroDivisionError:
#         val_balanced_accuracy =0
    try:
        val_precision_neg=tn/(tn+fn)
    except ZeroDivisionError:
        val_precision_neg=0
    try:
        val_recall_neg=tn/(tn+fp)
    except ZeroDivisionError:
        val_recall_neg=0
    try:
        val_f1_neg=2*val_precision_neg*val_recall_neg/(val_precision_neg+val_recall_neg)
    except ZeroDivisionError:
        val_f1_neg=0
    try:
        val_f1_beta_neg=((1+beta*beta)*val_precision_neg*val_recall_neg)/((1+beta*beta)*val_precision_neg*val_recall_neg+beta*beta*val_precision_neg+val_recall_neg)
    except ZeroDivisionError:
        val_f1_beta_neg=0
    try:
        val_f1_beta=(val_f1_beta_pos+val_f1_beta_neg)/2
    except ZeroDivisionError:
        val_f1_beta=0
        
    try:
        val_accuracy=(tp+tn)/(tp+tn+fp+fn)
    except ZeroDivisionError:
        val_accuracy=0
        print("ZeroDivisionError")

    return [val_loss, val_accuracy, val_precision_pos, val_recall_pos, val_f1_pos, val_f1_beta_pos,val_precision_neg, val_recall_neg, val_f1_neg, val_f1_beta_neg, val_f1_beta, val_total_not_predicted, pred_label] 
        
    
#     val_accuracy = np.mean(val_accuracy)



In [45]:
batch_size=[8, 12]
# learning_rate=[2e-5, 3e-5, 5e-5]
epochs=[1,2,3,4,5,6,7,8,9,10]
result_train=[[[] for bs in batch_size] for e in epochs]
result_val=[[[] for bs in batch_size] for e in epochs]
#result_val[e][lr][bs]=[metrics]

# Rule_mask_00

In [31]:
# import pickle
# print("val_loss, val_accuracy, val_precision_pos, val_recall_pos, val_f1_pos, val_f1_beta_pos, ...negs, f1_beta")
# best_model_val_acc=-1
# best_model_val_f1=-1
# best_model_val_f1_beta=-1
# for bs in range(len(batch_size)):
#     train_dataloader = torch.utils.data.DataLoader(
#         train_dataset_normal,
#         batch_size=batch_size[bs],
#         #shuffle=True
#     )
#     val_dataloader = torch.utils.data.DataLoader(
#         val_dataset,
#         batch_size=batch_size[bs],
#     )
# #     train_dataloader_MLM = torch.utils.data.DataLoader(
# #                     train_dataset,
# #                     batch_size=batch_size[bs],
# #                     #shuffle=True
# #     )
#     torch.cuda.empty_cache() 
#     model = BertForMaskedLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
#     model.to(device)
#     for e in range(len(epochs)): 
        
#         with open("prompt1_00_result_train.txt", "wb") as ft:   #Pickling
#             result_train[e][bs]=train(model, train_dataloader)
#             pickle.dump(result_train, ft)
#         ft.close()
#         with open("prompt1_00_result_val.txt", "wb") as fv:   #Pickling
#             result_val[e][bs]=evaluate(model, val_dataloader)
#             pickle.dump(result_val, fv)
#         fv.close()

#         print(f"batch_size: {batch_size[bs]} epoch: {epochs[e]} result_train: {result_train[e][bs]} result_val: {result_val[e][bs]}")
# #             if(e>1 and result_val[e][lr][bs][5]<result_val[e-1][lr][bs][5]):
# #                 print("Early stopping")
# #                 break
#         if result_val[e][bs][1] > best_model_val_acc:
#             best_model_val_acc=result_val[e][bs][1]
# #                 !rmdir model_prompt1_MLM
#             print(f"best_model_val_acc: {best_model_val_acc}")
#             model.save_pretrained(save_directory='./prompt1/model/microf1/prompt1_00/saved_model')
        
#         if result_val[e][bs][4] > best_model_val_f1:
#             best_model_val_f1=result_val[e][bs][4]
# #                 !rmdir model_prompt1_MLM
#             print(f"best_model_val_f1: {best_model_val_f1}")
#             model.save_pretrained(save_directory='./prompt1/model/f1/prompt1_00/saved_model')
#         if result_val[e][bs][10] > best_model_val_f1_beta:
#             best_model_val_f1_beta=result_val[e][bs][10]
# #                 !rmdir model_prompt1_MLM
#             print(f"best_model_val_f1_beta: {best_model_val_f1_beta}")
#             model.save_pretrained(save_directory='./prompt1/model/f1beta/prompt1_00/saved_model')

val_loss, val_accuracy, val_precision_pos, val_recall_pos, val_f1_pos, val_f1_beta_pos, ...negs, f1_beta


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


 check_train_accuracy: 0.9380981470169009
batch_size: 8 epoch: 1 result_train: [0.0362932507766043, 0.7361776925535315, 0.9349609765859516, 0.7465643975711089, 0.8302087960906263, 0.4709107769221465, 0.9413471502590673, 0.725790987535954, 0.8196336731931788, 0.4704980525399851, 0.47070441473106583, 2694] result_val: [0.004400055992934742, 0.506993006993007, 0.5035714285714286, 0.986013986013986, 0.6666666666666666, 0.3582317073170732, 0.6666666666666666, 0.027972027972027972, 0.053691275167785234, 0.10695187165775401, 0.2325917894874136, 0]
best_model_val_acc: 0.506993006993007
best_model_val_f1: 0.6666666666666666
best_model_val_f1_beta: 0.2325917894874136
 check_train_accuracy: 0.9867114953570285
batch_size: 8 epoch: 2 result_train: [0.00012286152706973444, 0.984819431128156, 0.9859379993608182, 0.9859379993608182, 0.9859379993608182, 0.4964596073382684, 0.987487969201155, 0.9837008628954937, 0.9855907780979828, 0.49665988963113566, 0.496559748484702, 24] result_val: [0.0094629783293

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


 check_train_accuracy: 0.916621134302262
batch_size: 12 epoch: 1 result_train: [0.052350147738435535, 0.6701821668264621, 0.9086101121218532, 0.6863215084691595, 0.7819754210286755, 0.4604318089234793, 0.9251808318264014, 0.6540428251837648, 0.7663358921550272, 0.4607262657871632, 0.46057903735532124, 3365] result_val: [0.00338371904520803, 0.5454545454545454, 0.5243445692883895, 0.9790209790209791, 0.6829268292682926, 0.3663003663003663, 0.8421052631578947, 0.11188811188811189, 0.19753086419753088, 0.26755852842809363, 0.31692944736422995, 0]
best_model_val_acc: 0.5454545454545454
best_model_val_f1: 0.6829268292682926
best_model_val_f1_beta: 0.31692944736422995
 check_train_accuracy: 0.9847683181016514
batch_size: 12 epoch: 2 result_train: [0.00018597814739200724, 0.9814637264301694, 0.9824505424377792, 0.9840204538191115, 0.9832348714673479, 0.49565357372826785, 0.9871092491137609, 0.9789069990412272, 0.9829910141206675, 0.496337827326938, 0.4959957005276029, 42] result_val: [0.00530

In [32]:
#######################################################

adjustment for the runtime error

In [33]:
#pick the best model
max_result=-1
for e in range(len(epochs)):
    for bs in range(len(batch_size)):
        if(len(result_val[e][bs])!=0):
            if(result_val[e][bs][1]>max_result):
                max_result=result_val[e][bs][1]
                max_e=epochs[e]
                max_bs=batch_size[bs]
                    
max_result,max_e,max_bs

(0.5699300699300699, 3, 12)

In [34]:
result_val[0][1]

[0.00338371904520803,
 0.5454545454545454,
 0.5243445692883895,
 0.9790209790209791,
 0.6829268292682926,
 0.3663003663003663,
 0.8421052631578947,
 0.11188811188811189,
 0.19753086419753088,
 0.26755852842809363,
 0.31692944736422995,
 0]

In [23]:
test_dataloader = torch.utils.data.DataLoader(
                test_dataset,
                batch_size=12,
                )

In [22]:
val_dataloader = torch.utils.data.DataLoader(
                val_dataset,
                batch_size=12,
                )

In [36]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/model/microf1/prompt1_00/saved_model")
model.to(device)
max_result_test=evaluate(model, test_dataloader)
print(f"batch_size: {max_bs} epoch: {max_e} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.0056376808249216075, 0.5936132983377078, 0.5719825496849249, 0.9624796084828712, 0.7175433262389784, 0.38366497593965404, 0.7937219730941704, 0.1669811320754717, 0.27591582229150424, 0.31194924215720826, 0.34780710904843115, 0]


In [25]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/microf1/prompt1_00/saved_model")
model.to(device)
prob_tp=[]
prob_fp=[]
prob=[]
max_result_test=evaluate(model, val_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.006133526012386407, 0.5699300699300699, 0.5390625, 0.965034965034965, 0.6917293233082706, 0.37156704361873993, 0.8333333333333334, 0.17482517482517482, 0.2890173410404624, 0.32216494845360827, 0.3468659960361741, 0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [24]:
print(len(prob_tp))
print(len(prob_fp))
tp=len(prob_tp)
fp=len(prob_fp)
precision=tp/(tp+fp)
recall=tp/143
f1=2*precision*recall/(precision+recall)
print(precision)
print(recall)
print(f1)

138
118
0.5390625
0.965034965034965
0.6917293233082706


In [29]:
prob_tp

[0.999987006187439,
 0.9983225464820862,
 0.999991774559021,
 0.9982933402061462,
 0.9998859167098999,
 0.999930739402771,
 0.9994188547134399,
 0.9997108578681946,
 0.9999858140945435,
 0.9999589920043945,
 0.9987497329711914,
 0.9864532351493835,
 0.9999954700469971,
 0.9916287660598755,
 0.9999717473983765,
 0.9999109506607056,
 0.9991077780723572,
 0.99982088804245,
 0.9981914162635803,
 0.9999964237213135,
 0.8956517577171326,
 0.9916883111000061,
 0.9999438524246216,
 0.999950647354126,
 0.9953450560569763,
 0.9984654188156128,
 0.9803815484046936,
 0.9783726334571838,
 0.9938368201255798,
 0.9996391534805298,
 0.9987803101539612,
 0.9999175071716309,
 0.9120179414749146,
 0.9998760223388672,
 0.9999970197677612,
 0.9999099969863892,
 0.9850530028343201,
 0.8488094210624695,
 0.9924578666687012,
 0.9999914169311523,
 0.9997753500938416,
 0.9931100606918335,
 0.9979475140571594,
 0.9979378581047058,
 0.9996160268783569,
 0.9979070425033569,
 0.9991681575775146,
 0.9993554949760437

In [42]:
tp=0
fp=0
for i in range(len(prob_tp)):
    if prob_tp[i]>0.9999758 :
        tp+=1
for i in range(len(prob_fp)):
    if prob_fp[i]>0.9999758 :
        fp+=1
print(tp)
print(fp)
precision=tp/(tp+fp)
recall=tp/143
f1=2*precision*recall/(precision+recall)
print(precision)
print(recall)
print(f1)

28
12
0.7
0.1958041958041958
0.30601092896174864


In [30]:
prob_fp

[0.9999370574951172,
 0.9997527003288269,
 0.9999934434890747,
 0.9999828338623047,
 0.9875956773757935,
 0.9994445443153381,
 0.999606192111969,
 0.9971023201942444,
 0.999680757522583,
 0.6370437741279602,
 0.9997876286506653,
 0.9998310804367065,
 0.9995880722999573,
 0.9999701976776123,
 0.9974743723869324,
 0.9999610185623169,
 0.9985907673835754,
 0.9998738765716553,
 0.9995265007019043,
 0.9999594688415527,
 0.9988705515861511,
 0.9993197917938232,
 0.9999912977218628,
 0.9998747110366821,
 0.9982275366783142,
 0.9998612403869629,
 0.9987413287162781,
 0.9999805688858032,
 0.9998881816864014,
 0.9997420907020569,
 0.9999909400939941,
 0.9971489310264587,
 0.9930790066719055,
 0.9987659454345703,
 0.9672781825065613,
 0.9878236055374146,
 0.999886155128479,
 0.9999197721481323,
 0.998554527759552,
 0.9999817609786987,
 0.997901201248169,
 0.9973033666610718,
 0.9265697002410889,
 0.9991607666015625,
 0.997066080570221,
 0.9999938011169434,
 0.9988343119621277,
 0.9981493949890137

-------------------------------------------------

In [69]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/microf1/prompt1_00/saved_model")
model.to(device)
prob_tp=[]
prob_fp=[]
prob=[]
max_result_noisy_train=evaluate(model, noisy_train_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_noisy_train[:-1]}")

batch_size: 12 epoch: 3 result_test: [0.0008179426875929482, 0.9171312584573749, 1.0, 0.9171312584573749, 0.9567746124961178, 0.4955226324932298, 0.0, 0, 0, 0, 0.2477613162466149, 0]


In [27]:
import pickle
with open("val_labels/prompt1_00_labels.txt", "wb") as f:   #Pickling
    prompt1_labels=max_result_test[-1]
    pickle.dump(prompt1_labels, f)
f.close()

# 2000eval

In [27]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/f1/prompt1_25/saved_model")
model.to(device)
prob=[]
max_result_test=evaluate(model, test_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.0027303161439998216, 0.5990990990990991, 0.5768386388583974, 0.9722479185938946, 0.7240785394419567, 0.38571638285378745, 0.8295454545454546, 0.15921483097055616, 0.26715462031107046, 0.3105061675882604, 0.3481112752210239, 2, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [24]:
import pickle
with open("2000/test_labels/prompt1_25_labels.txt", "wb") as f:   #Pickling
    prompt1_labels=max_result_test[-1]
    pickle.dump(prompt1_labels, f)
f.close()

In [28]:
import pickle
with open("2000/test_labels/prompt1_25_prob.txt", "wb") as f:   #Pickling
    pickle.dump(prob, f)
f.close()

In [31]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/f1/prompt1_25/saved_model")
model.to(device)
prob=[]
max_result_test=evaluate(model, val_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.00338371904520803, 0.5454545454545454, 0.5243445692883895, 0.9790209790209791, 0.6829268292682926, 0.3663003663003663, 0.8421052631578947, 0.11188811188811189, 0.19753086419753088, 0.26755852842809363, 0.31692944736422995, 0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1

In [32]:
import pickle
with open("2000/val_labels/prompt1_25_labels.txt", "wb") as f:   #Pickling
    prompt1_labels=max_result_test[-1]
    pickle.dump(prompt1_labels, f)
f.close()

In [33]:
import pickle
with open("2000/val_labels/prompt1_25_prob.txt", "wb") as f:   #Pickling
    pickle.dump(prob, f)
f.close()

In [23]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/f1/prompt1_00/saved_model")
model.to(device)
prob=[]
max_result_test=evaluate(model, test_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.005576186052717085, 0.597, 0.5766463752075263, 0.9621421975992613, 0.7211072664359862, 0.38532652910287696, 0.7875647668393783, 0.16575790621592149, 0.2738738738738739, 0.31033074724377296, 0.347828638173325, 0, [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [24]:
import pickle
with open("2000/test_labels/prompt1_00_labels.txt", "wb") as f:   #Pickling
    prompt1_labels=max_result_test[-1]
    pickle.dump(prompt1_labels, f)
f.close()

In [25]:
import pickle
with open("2000/test_labels/prompt1_00_prob.txt", "wb") as f:   #Pickling
    pickle.dump(prob, f)
f.close()

In [30]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/f1/prompt1_00/saved_model")
model.to(device)
prob=[]
max_result_test=evaluate(model, val_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.006133526012386407, 0.5699300699300699, 0.5390625, 0.965034965034965, 0.6917293233082706, 0.37156704361873993, 0.8333333333333334, 0.17482517482517482, 0.2890173410404624, 0.32216494845360827, 0.3468659960361741, 0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [27]:
import pickle
with open("2000/val_labels/prompt1_00_labels.txt", "wb") as f:   #Pickling
    prompt1_labels=max_result_test[-1]
    pickle.dump(prompt1_labels, f)
f.close()

In [28]:
import pickle
with open("2000/val_labels/prompt1_00_prob.txt", "wb") as f:   #Pickling
    pickle.dump(prob, f)
f.close()

Connor

In [24]:
torch.cuda.empty_cache() 
model = BertForMaskedLM.from_pretrained("../../prompt1/prompt1/model/f1/prompt1_25/saved_model")
model.to(device)
prob=[]
max_result_test=evaluate(model, test_dataloader)
print(f"batch_size: {12} epoch: {3} result_test: {max_result_test}")

batch_size: 12 epoch: 3 result_test: [0.00391549773566011, 0.43, 0.37362637362637363, 1.0, 0.544, 0.2992957746478873, 1.0, 0.13636363636363635, 0.24000000000000002, 0.30612244897959184, 0.3027091118137396, 0, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]]
